In [1]:
from imutils import paths
import face_recognition
import pickle
import cv2
import os
import smtplib
import xlsxwriter
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from openpyxl import load_workbook, Workbook
from datetime import date 
import imutils
import time

In [ ]:
gmail_user = 'Your Email'
gmail_password = 'PW'

In [ ]:
sent_from = gmail_user
subject = 'school attendance'

In [ ]:
today = date.today().strftime('%Y-%m-%d')

In [ ]:
imagePaths = list(paths.list_images('image_path'))
knownEncodings = []
knownNames = []
# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    # extract the person name from the image path
    name = imagePath.split(os.path.sep)[-2]
    # load the input image and convert it from BGR (OpenCV ordering)
    # to dlib ordering (RGB)
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #Use Face_recognition to locate faces
    boxes = face_recognition.face_locations(rgb,model='hog')
    print(boxes)
    # compute the facial embedding for the face
    encodings = face_recognition.face_encodings(rgb, boxes)
    # loop over the encodings
    for encoding in encodings:
        knownEncodings.append(encoding)
        knownNames.append(name)
#save emcodings along with their names in dictionary data
data = {"encodings": knownEncodings, "names": knownNames}

#use pickle to save data into a file for later use
f = open("face_enc", "wb")
f.write(pickle.dumps(data))
f.close()

In [ ]:
#find path of xml file containing haarcascade file 
cascPathface = os.path.dirname(
 cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
# load the harcaascade in the cascade classifier
faceCascade = cv2.CascadeClassifier(cascPathface)
# load the known faces and embeddings saved in last file
data = pickle.loads(open('face_enc', "rb").read())

In [ ]:
# open info file where the email
info = pd.read_csv('info.csv')

In [ ]:
# loop over frames from the video file stream
#print("Streaming started")
video_capture = cv2.VideoCapture(0)
while True:
    # grab the frame from the threaded video stream
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    # convert the input frame from BGR to RGB 
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # the facial embeddings for face in input
    encodings = face_recognition.face_encodings(rgb)
    names = []
    # loop over the facial embeddings incase
    # we have multiple embeddings for multiple fcaes
    for encoding in encodings:
       #Compare encodings with encodings in data["encodings"]
       #Matches contain array with boolean values and True for the embeddings it matches closely
       #and False for rest
        matches = face_recognition.compare_faces(data["encodings"],
         encoding)
        #set name =inknown if no encoding matches
        name = "Unknown"
        # check to see if we have found a match
        if True in matches:
            #Find positions at which we get True and store them
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            # loop over the matched indexes and maintain a count for
            # each recognized face face
            for i in matchedIdxs:
                #Check the names at respective indexes we stored in matchedIdxs
                name = data["names"][i]
                #increase count for the name we got
                counts[name] = counts.get(name, 0) + 1
            #set name which has highest count
            name = max(counts, key=counts.get)
            
            # updating the column value/data
            if name not in df['name'].values:
                print(name)
                row = len(df)
                df.loc[row, :] = [name, datetime.now()]
                
                #df.to_csv("attendance.csv", index=False)
                #save in Excel
                df.to_excel("attendance_1.xlsx", sheet_name= today)
                
                # send an email
                to = info[info['name'] == name]['email'].values[0]
                print(to)
        
                
                body = f'Dear parent,\n We would like to inform you that your child {name} has arrived at school on {datetime.now()} \n Kind Regards\n Your school Team'
                message = "Subject: {}\n\n{}".format(subject, body)
                
                try:
                    smtp_server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
                    smtp_server.ehlo()
                    smtp_server.login(gmail_user, gmail_password)
                    smtp_server.sendmail(sent_from, to, message)
                    smtp_server.close()
                    print ("Email sent successfully!")
                except Exception as ex:
                        print ("Something went wrong….",ex)
                
                #print(df.tail(1))
            
 
 
        # update the list of names
        names.append(name)
        #print(names)
        # loop over the recognized faces
        for ((x, y, w, h), name) in zip(faces, names):
            # rescale the face coordinates
            # draw the predicted face name on the image
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
             0.75, (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()            